# Definição do Markov Decision Problem (MDP)

In [ ]:
import numpy as np
np.set_printoptions(precision=2, suppress=True)

# States
S = ['1', '2', '3']

# Actions
A = ['L', 'R']

# Transition probabilities
L = np.array([[1.0, 0.0, 0.0],
              [0.8, 0.2, 0.0],
              [0.0, 0.8, 0.2]])

R = np.array([[0.2, 0.8, 0.0],
              [0.0, 0.2, 0.8],
              [0.0, 0.0, 1.0]])

P = [L, R]

# Reward function
R = np.array([[0.0, 0.0],
              [0.0, 0.0],
              [1.0, 1.0]])

gamma = 0.9


# Value Iteration 

In [ ]:
# Initialize V
V = np.ones(len(S))

err = 1
i = 0

while err > 1e-2:
    Q = []

    # Compute Q-values associated with each action
    for a in range(len(A)):
        Q += [R[:, a] + gamma * P[a].dot(V)]
    print('Q values at time', i)
    print(Q)

    # Compute maximum for each state
    Vnew = np.max(Q, axis=0)
    print('V-function at time', i)
    print(Vnew)

    # Compute error
    err = np.linalg.norm(V - Vnew)

    # Update V
    V = Vnew

    i += 1

print('\nV* =')
print(V[:, None])

print('\nQ* =')
print(Q)


# Função de avaliação de política

In [ ]:
def evaluate(Rw, P, pol):
    """
    evaluate(Rw, P, pol) computes the state–value function associated with policy pol.

    :param Rw: 2D np.ndarray with |S| rows and |A| columns; element Rw[s,a] is the reward of action a in state s
    :param P: 3D np.ndarray with |A| |S| x |S| matrices; matrix P[a] is the transition matrix for action a
    :param pol: 2D np.ndarray with same dimensions as Rw; element pol[s,a] is the probability of action a in state s
    :return V: 2D np.ndarray with |S| elements, where element s is Vpi(s).
    """
    # Problem dimensions
    nS, nA = Rw.shape

    # Policy-averaged reward
    Rpi = (pol * Rw).sum(axis=1)
    print('Rpi =')
    print(Rpi)

    # Policy-averaged probabilities
    Ppi = pol[:, 0, None] * P[0]
    for a in range(1, nA):
        Ppi += pol[:, a, None] * P[a]

    print('Ppi =')
    print(Ppi)

    # Use matrix inversion to compute Vpi
    Vpi = np.linalg.inv(np.eye(nS) - gamma * Ppi).dot(Rpi)

    return Vpi[:, None]
# -- End: evaluate


# Laço principal da Policy Iteration

In [ ]:
import time

# Initialize policy (uniform random)
pol = np.ones((len(S), len(A))) / len(A)

print('Initial policy =')
print(pol)

# Auxiliary matrix to store temporary Q-values
Q = np.zeros((len(S), len(A)))

quit = False
i = 0

t = time.time()

while not quit:
    # Evaluate policy
    V = evaluate(Rw, P, pol)
    print('V =')
    print(V)

    # Compute Q-values
    for a in range(len(A)):
        Q[:, a, None] = Rw[:, a, None] + gamma * P[a].dot(V)

    print('Q =')
    print(Q)

    # Compute maximizing policy
    Qmax = Q.max(axis=1, keepdims=True)
    polnew = np.isclose(Q, Qmax, atol=1e-10, rtol=1e-10).astype(int)
    polnew = polnew / polnew.sum(axis=1, keepdims=True)

    print('Policy =')
    print(polnew)

    # Check for convergence
    quit = (pol == polnew).all()
    pol = polnew
    i += 1

t = time.time() - t

print('N. iterations:', i)
print('Time taken:', round(t, 3), 'seconds')
print('\npi* =')
print(pol)
print('\nV* =')
print(V[:, None])
